In [ ]:
!pip install pymysql
%load_ext sql
!rm -rf ejemplos.db
%sql sqlite:///ejemplos.db

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.1 MB/s eta 0:00:00


Ejercicio 1:

In [ ]:
%%sql

CREATE TABLE Estudiantes (
id_estudiante INT PRIMARY KEY,
nombre VARCHAR(50)
);
CREATE TABLE Cursos (
id_curso INT PRIMARY KEY,
nombre VARCHAR(50)
);
CREATE TABLE Matriculas (
id_matricula INT PRIMARY KEY,
id_estudiante INT,
id_curso INT,
fecha DATE,
FOREIGN KEY (id_estudiante) REFERENCES Estudiantes(id_estudiante),
FOREIGN KEY (id_curso) REFERENCES Cursos(id_curso)
);


 * sqlite:///ejemplos.db
Done.
Done.


[]

Opciones para evitar violaciones a la integridad referencial.

In [ ]:
%%sql
FOREIGN KEY (id_estudiante) REFERENCES Estudiantes(id_estudiante) ON DELETE CASCADE
FOREIGN KEY (id_estudiante) REFERENCES Estudiantes(id_estudiante) ON DELETE SET NULL

Ejercicio 2:

In [ ]:
%%sql
INSERT INTO Cursos (id_curso, nombre_curso) VALUES (1, 'Matemáticas');
INSERT INTO Cursos (id_curso, nombre_curso) VALUES (2, 'Historia');
INSERT INTO Matriculas (id_matricula, id_estudiante, id_curso) VALUES (1, 999, 1);

**Mensaje de Error:**
ERROR 1452 (23000): Cannot add or update a child row: a foreign key constraint fails
(basedatos.Matriculas, CONSTRAINT Matriculas_ibfk_1 FOREIGN KEY (id_estudiante)
REFERENCES Estudiantes (id_estudiante))

Ejercicio 3:

In [ ]:
%%sql

CREATE TABLE Cuentas (
    id_cuenta INT PRIMARY KEY,
    saldo DECIMAL(10, 2)
);
INSERT INTO Cuentas (id_cuenta, saldo) VALUES (1, 500.00);

READ COMMITED:

In [ ]:
%%sql
--Primer usuario
START TRANSACTION;
SELECT saldo FROM Cuentas WHERE id_cuenta = 1;
UPDATE Cuentas SET saldo = saldo - 100 WHERE id_cuenta = 1;
COMMIT;
--Segundo usuario
START TRANSACTION;
SELECT saldo FROM Cuentas WHERE id_cuenta = 1;
UPDATE Cuentas SET saldo = saldo - 50 WHERE id_cuenta = 1;
COMMIT;

SERIALIZABLE:

In [ ]:
%%sql
-- Primer usuario
SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;
BEGIN;
SELECT saldo FROM Cuentas WHERE id_cuenta = 1;
UPDATE Cuentas SET saldo = saldo - 100 WHERE id_cuenta = 1;
--Segundo usuario
SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;
BEGIN;
SELECT saldo FROM Cuentas WHERE id_cuenta = 1;
UPDATE Cuentas SET saldo = saldo - 100 WHERE id_cuenta = 1;
COMMIT;



* **Read commited**: Ambas transacciones verán el mismo saldo inicial. La 2datransacción no
vera los cambios realizados por la 1ra. El saldo será el mismo en ambos casos, pero una de
las extracciones no queda reflejada.
*  **Serializable**: Las transacciones se ejecutan una después de otra, pero la base de datos
evita que lean el mismo saldo simultáneamente. Por lo tanto, la 2da operación fallara al tatar
de hacer el commit.




Ejercicio 4:

Sin indices

In [ ]:
%%sql
EXPLAIN SELECT * FROM productos WHERE marca = 'Stark';

Al utilizar full table scan (sin índices) se tardo 0.1 segundos.

Con indices:

In [ ]:
%%sql
CREATE INDEX idx_marca ON productos(marca);
EXPLAIN SELECT * FROM productos WHERE marca = 'Stark';

Se pudo ver que el tiempo de lectura fue menor al que se obtuvo sin la utilización de
índices, ya que tardo 0.016 segundos.

Ejercicio 6:

Creamos la vista

In [ ]:
%%sql
CREATE VIEW ventas_mensuales AS
SELECT
    id AS producto_id,
    nombre,
    DATE_FORMAT(fecha_creacion, '%Y-%m') AS mes,
    stock AS total_vendido
FROM productos;

Ejecutamos la consulta


In [ ]:
%%sql
SELECT
    producto_id,
    nombre,
    SUM(total_vendido) AS total_ventas
FROM ventas_mensuales
GROUP BY producto_id, nombre
ORDER BY total_ventas DESC
LIMIT 5;

Ejercicio 7:

In [ ]:
%%sql
CREATE USER 'analista'@'localhost' IDENTIFIED BY 'analista';

GRANT SELECT ON BaseDeDatos.productos TO 'analista'@'localhost';


En el bash

In [ ]:
mysql -u analista -p

Probar desde el usuario analista

In [ ]:
%%sql
INSERT INTO productos (id, nombre, descripcion, precio, stock, categoria, marca, fecha_creacion)
VALUES (97, 'NombProd', 'desc', 68, 1, 'cat', 'marca', '2025-07-16');

Al tratar de insertar con el usuario salta el siguiente mensaje:
**ERROR 1142 (42000): INSERT command denied to user 'usuario'@'localhost'
for table 'productos'**
Ya que el usuario solo tiene permiso de SELECT, y no puede ejecutar comandos como
insert.

Ejercicio 8:

In [ ]:
%%sql
CREATE TABLE AuditoriaClientes (
    id_auditoria INT PRIMARY KEY AUTO_INCREMENT,
    id_cliente INT,
    operacion VARCHAR(10),
    fecha TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    datos_anterior TEXT,
    datos_nuevo TEXT
);

In [ ]:
%%sql
CREATE TRIGGER before_cliente_update
BEFORE UPDATE ON Clientes
FOR EACH ROW
BEGIN
    INSERT INTO AuditoriaClientes (id_cliente, operacion, datos_anterior, datos_nuevo)
    VALUES (OLD.id_cliente, 'UPDATE', OLD.nombre, NEW.nombre);
END;

CREATE TRIGGER before_cliente_delete
BEFORE DELETE ON Clientes
FOR EACH ROW
BEGIN
    INSERT INTO AuditoriaClientes (id_cliente, operacion, datos_anterior)
    VALUES (OLD.id_cliente, 'DELETE', OLD.nombre);
END;

CREATE TRIGGER after_cliente_insert
AFTER INSERT ON Clientes
FOR EACH ROW
BEGIN
    INSERT INTO AuditoriaClientes (id_cliente, operacion, datos_nuevo)
    VALUES (NEW.id_cliente, 'INSERT', NEW.nombre);
END;

Ejercicio 9:

Crea copia de seguridad

In [ ]:
mysqldump -u $DB_USER -p$DB_PASSWORD $DB_NAME >$BACKUP_FILE


Hacer el script ejecutable

In [ ]:
chmod +x backup_mysql.sh

Configurar el Cron Job para el Backup Diario


In [ ]:
crontab –e

Añade la siguiente línea al final del archivo para ejecutar el script a las 2:00
AM todos los días:

In [ ]:
0 2 * * * /ruta/a/tu/script/backup_mysql.sh

Restaurar el Backup

In [ ]:
mysql -u root -p tu_base_de_datos <
/ruta/a/tu/carpeta/backups/backup_tu_base_de_datos_2025-04-21.sql

Simulación de pérdida de datos y recuperación


1. Borrar (por error o intencionalmente) una o más tablas/filas de la base de
datos.
2. Restaurar desde el archivo de respaldo usando el comando anterior.
3.Verificar que la información haya vuelto correctamente.

